# Limpieza datos combinados Usuarios y Ordenes de trabajo Zentry - *Piloto Pacífico*

Se genera combinación de datos propios de los usuario, con ordenes de trabajo (odt) para años 2022-2024 y se cuenta con anomalías confirmadas hasta 2023.
Estas últimas permitirán marcar el caso positivo de anomalías y las demás ordenes serán nuestro negativo de anomalía. Tomando el caso como una regresión logistica multivariado.

Dicho lo anterior, debido al corte de fecha de las anomalías confirmadas se tendrá como muestra total las odt de 2022 y 2023 - Zona Pacífico.

In [87]:
import os

import numpy as np
import pandas as pd

from cedenar_anomalies.utils.paths import (
    data_processed_dir,
    data_raw_dir,
    reports_dir
)

In [3]:
os.getcwd()

'/Users/frandak2/Documents/repos/cedenar_anomalies/notebooks'

In [4]:
pd.set_option("display.max_rows", None)  # Mostrar todas las filas
pd.set_option("display.max_columns", None)  # Mostrar todas las columnas

## Datos por año

#### 2023

In [5]:
# ruta_archivo = "J:\\JESSICA\\Jessica\\Rafael\\igrid\\cedenar\\python_zentr\\dataset_decantado_2023.csv"
ruta_archivo = data_processed_dir("dataset_decantado_2023.csv")
df_2023 = pd.read_csv(ruta_archivo)
df_2023 = df_2023[df_2023.ZONA == 'PACIFICO'].copy()

/var/folders/pq/cm_j30pd6d5gvwryc4gttm2h0000gn/T/ipykernel_9627/1848789.py:3: DtypeWarning: Columns (54) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2023 = pd.read_csv(ruta_archivo)


In [6]:
filas, columnas = df_2023.shape
filas

13502

## Duplicados

In [7]:
def revisar_y_guardar_duplicados(df, nombre_df):
    """
    Revisa si hay duplicados en las columnas 'orden' y 'odt' de un DataFrame y guarda los duplicados en un archivo CSV.

    Parámetros:
    - df (DataFrame): El DataFrame a revisar.
    - nombre_df (str): El nombre del DataFrame para usar en el nombre del archivo.
    """

    # Identificar duplicados basados en 'orden' y 'odt'
    duplicados = df.duplicated(subset=["orden", "odt"], keep=False)

    # Contar el número de duplicados
    num_duplicados = duplicados.sum()

    # Guardar los duplicados en un archivo CSV si existen
    if num_duplicados > 0:
        duplicados_df = df[duplicados]
        nombre_archivo = f"{nombre_df}_duplicados.csv"
        duplicados_df.to_csv(nombre_archivo, index=False)
        print(
            f"Se encontraron {num_duplicados} registros duplicados en {nombre_df} y se guardaron en {nombre_archivo}."
        )
    else:
        print(f"No se encontraron registros duplicados en {nombre_df}.")

In [8]:
# Revisar y guardar duplicados en df_2023
revisar_y_guardar_duplicados(df_2023, "df_2023")

No se encontraron registros duplicados en df_2023.


In [9]:
filas, columnas = df_2023.shape
print(f"Registros 2023 - duplicados = {filas - 0}")

Registros 2023 - duplicados = 13502


### Eliminar duplicados

In [10]:
def eliminar_duplicados(df, nombre_df):
    """
    Elimina registros duplicados en las columnas 'orden' y 'odt' de un DataFrame, conservando solo la primera aparición.

    Parámetros:
    - df (DataFrame): El DataFrame a procesar.
    - nombre_df (str): El nombre del DataFrame para imprimir en los resultados.
    """

    # Eliminar duplicados, conservando la primera aparición
    df_sin_duplicados = df.drop_duplicates(subset=["orden", "odt"], keep="first")

    # Imprimir información sobre el DataFrame resultante
    print(
        f"Registros duplicados eliminados en {nombre_df}. Tamaño original: {len(df)}, Tamaño después de eliminar duplicados: {len(df_sin_duplicados)}"
    )

    return df_sin_duplicados

In [11]:
# Eliminar duplicados en df_2023
df_2023 = eliminar_duplicados(df_2023, "df_2023")

df_2023.to_csv("df_2023_sin_duplicados.csv", index=False)

print("DataFrames sin duplicados guardados en archivos CSV.")

Registros duplicados eliminados en df_2023. Tamaño original: 13502, Tamaño después de eliminar duplicados: 13502
DataFrames sin duplicados guardados en archivos CSV.


## Variables a prescindir

### Ubicación

In [12]:
# Guardar el nuevo dataframe en un archivo csv
# Eliminar los registros con valores nulos en 'LATI_USU'
df_2023 = df_2023.dropna(subset=["LATI_USU"])
df_2023.to_csv("df_2023_sin_nulos.csv", index=False)

### Inspección Estado Del Predio

In [13]:
print(df_2023["item_237"].unique())

[True nan]


In [14]:
df_2023["item_237"].unique()

array([True, nan], dtype=object)

In [15]:
# Reemplazar NaN con False en df_2023['item_237']
df_2023["item_237"] = df_2023["item_237"].replace(np.nan, False)

/var/folders/pq/cm_j30pd6d5gvwryc4gttm2h0000gn/T/ipykernel_9627/3629671584.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_2023["item_237"] = df_2023["item_237"].replace(np.nan, False)


In [16]:
print(df_2023["item_237"].value_counts())

item_237
True     11769
False     1733
Name: count, dtype: int64


### Medida Activa

In [17]:
print(df_2023["item_23"].unique())

['SI' nan 'TRUE']


In [18]:
df_2023["item_23"] = df_2023["item_23"].replace({np.nan: "NO", "TRUE": "SI"})

In [19]:
print(df_2023["item_23"].value_counts())

item_23
SI    10844
NO     2658
Name: count, dtype: int64


### Medida Reactiva

In [20]:
print(df_2023["item_24"].unique())

['NO' nan 'SI' 'TRUE']


In [21]:
df_2023["item_24"] = df_2023["item_24"].replace({np.nan: "NO", "TRUE": "SI"})

In [22]:
print(df_2023["item_24"].value_counts())

item_24
NO    8452
SI    5050
Name: count, dtype: int64


### Tipo Sello 1 Tapa Bornera

In [23]:
print(df_2023["item_601"].unique())

['DA' nan 'DT' 'RA' 'DY' 'TR' 'FA' 'AZ' 'AO' 'CF' 'VT' 'VR' 'VE' 'NE' 'AT'
 'BT' 'RN' 'EP' 'GR' 'BL' 'TA' 'AL']


In [24]:
df_2023["item_601"] = df_2023["item_601"].replace(np.nan, "NOAPL")

In [25]:
conteo_nan = df_2023["item_601"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0


### Sello 1 Tapa Principal

In [26]:
conteo_nan = df_2023["item_33"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 3898


In [27]:
df_2023["item_33"] = df_2023["item_33"].replace(np.nan, "NO INDICA")

In [28]:
conteo_nan = df_2023["item_33"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0


### Tipo Sello 1 Tapa Principal

In [29]:
conteo_nan = df_2023["item_598"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 3864


In [30]:
df_2023["item_598"] = df_2023["item_598"].replace(np.nan, "NOAPL")

In [31]:
conteo_nan = df_2023["item_598"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0


### Sello 2 Tapa Principal

In [32]:
conteo_nan = df_2023["item_35"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 5272


In [33]:
df_2023["item_35"] = df_2023["item_35"].replace(np.nan, "NO INDICA")

In [34]:
conteo_nan = df_2023["item_35"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0


### Tipo Sello 2 Tapa Principal

In [35]:
conteo_nan = df_2023["item_599"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 5285


In [36]:
df_2023["item_599"] = df_2023["item_599"].replace(np.nan, "NOAPL")

In [37]:
conteo_nan = df_2023["item_599"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0


### Tipo Sello Caja

In [38]:
conteo_nan = df_2023["item_603"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 5669


In [39]:
df_2023["item_603"] = df_2023["item_603"].replace(np.nan, "NOAPL")

In [40]:
conteo_nan = df_2023["item_603"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0


### Sello Caja

In [41]:
conteo_nan = df_2023["item_43"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 5712


In [42]:
df_2023["item_43"] = df_2023["item_43"].replace(np.nan, "NO INDICA")

In [43]:
conteo_nan = df_2023["item_43"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0


### Corriente (A)

In [44]:
conteo_nan = df_2023["item_108"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 5522


In [45]:
df_2023["item_108"] = df_2023["item_108"].replace(np.nan, "NO INDICA")

In [46]:
conteo_nan = df_2023["item_108"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0


### Tipo Sello 2 Tapa Bornera

In [47]:
conteo_nan = df_2023["item_602"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 8892


In [48]:
df_2023["item_602"] = df_2023["item_602"].replace(np.nan, "NOAPL")

In [49]:
conteo_nan = df_2023["item_602"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0


### Tipo Sello Home Display

In [50]:
conteo_nan = df_2023["item_597"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 11037


In [51]:
df_2023["item_597"] = df_2023["item_597"].replace(np.nan, "NOAPL")

In [52]:
conteo_nan = df_2023["item_597"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0


### Sin Sello O Sello Abierto En Caja

In [53]:
print(df_2023["item_74"].unique())

[nan True]


In [54]:
# Reemplazar NaN con False en df_2023['item_237']
df_2023["item_74"] = df_2023["item_74"].replace(np.nan, False)

/var/folders/pq/cm_j30pd6d5gvwryc4gttm2h0000gn/T/ipykernel_9627/1211262664.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_2023["item_74"] = df_2023["item_74"].replace(np.nan, False)


In [55]:
print(df_2023["item_74"].value_counts())

item_74
False    12134
True      1368
Name: count, dtype: int64


### Tipo Acometida

In [56]:
print(df_2023["item_248"].value_counts())

item_248
INSTALACION ACOM MONOFASICA    447
INSTALACION ACOM BIFASICA       47
INSTALACION ACOM TRIFASICA      19
Name: count, dtype: int64


In [57]:
conteo_nan = df_2023["item_248"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 12989


In [58]:
df_2023["item_248"] = df_2023["item_248"].replace(np.nan, "NO INDICA")

In [59]:
conteo_nan = df_2023["item_248"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0


### Sin Sello (S) Tapa Principal

In [60]:
conteo_nan = df_2023["item_68"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 13487


In [61]:
df_2023["item_68"] = df_2023["item_68"].replace(np.nan, False)

/var/folders/pq/cm_j30pd6d5gvwryc4gttm2h0000gn/T/ipykernel_9627/2267857441.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_2023["item_68"] = df_2023["item_68"].replace(np.nan, False)


In [62]:
conteo_nan = df_2023["item_68"].isna().sum()
print(f"Número de valores NaN: {conteo_nan}")

Número de valores NaN: 0


In [63]:
df_2023["item_68"].unique()

array([False,  True])

### Descripcion

*Anomalias confirmadas*

In [64]:
df_2023["Descripcion"].unique()

array([nan, 'Anomalia verificada y aprobada',
       'Anomalia verificada con acta digital', 'Posible anomalia'],
      dtype=object)

In [65]:
df_2023["Descripcion"].unique()

array([nan, 'Anomalia verificada y aprobada',
       'Anomalia verificada con acta digital', 'Posible anomalia'],
      dtype=object)

In [66]:
print(df_2023["Descripcion"].value_counts())

Descripcion
Anomalia verificada y aprobada          1616
Anomalia verificada con acta digital     677
Posible anomalia                         107
Name: count, dtype: int64


In [67]:
df_2023["Descripcion"] = df_2023["Descripcion"].replace(np.nan, "Solo odt")

In [68]:
print(df_2023["Descripcion"].value_counts())

Descripcion
Solo odt                                11102
Anomalia verificada y aprobada           1616
Anomalia verificada con acta digital      677
Posible anomalia                          107
Name: count, dtype: int64


## Dataframes limpios

In [69]:
# Lista de columnas a conservar
columnas_a_conservar = [
    "item_68",
    "item_248",
    "item_74",
    "item_597",
    "item_602",
    "item_108",
    "item_43",
    "item_603",
    "item_599",
    "item_35",
    "item_598",
    "item_33",
    "item_601",
    "item_24",
    "item_23",
    "item_237",
    "LATI_USU",
    "LONG_USU",
    "NIVEL",
    "AREA",
    "item_288",
    "odt",
    "orden",
    "PLAN_COMERCIAL",
    "Descripcion",
]

df_2023_filtrado = df_2023[columnas_a_conservar]

# Guardar los DataFrames filtrados en archivos CSV
df_2023_filtrado.to_csv("df_2023_filtrado.csv", index=False)

print("\nDataFrames filtrados guardados en archivos CSV.")


DataFrames filtrados guardados en archivos CSV.


In [70]:
print("df_2023_filtrado:")
df_2023_filtrado.info()

df_2023_filtrado:
<class 'pandas.core.frame.DataFrame'>
Index: 13502 entries, 5 to 66367
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   item_68         13502 non-null  bool   
 1   item_248        13502 non-null  object 
 2   item_74         13502 non-null  bool   
 3   item_597        13502 non-null  object 
 4   item_602        13502 non-null  object 
 5   item_108        13502 non-null  object 
 6   item_43         13502 non-null  object 
 7   item_603        13502 non-null  object 
 8   item_599        13502 non-null  object 
 9   item_35         13502 non-null  object 
 10  item_598        13502 non-null  object 
 11  item_33         13502 non-null  object 
 12  item_601        13502 non-null  object 
 13  item_24         13502 non-null  object 
 14  item_23         13502 non-null  object 
 15  item_237        13502 non-null  bool   
 16  LATI_USU        13502 non-null  float64
 17  LONG_USU        13

## Marcar reincidencias

In [71]:
def crear_campo_reincidente(df):
    """
    Crea el campo 'reincidente' en un DataFrame basado en la columna 'item_288'.

    Parámetros:
    - df (DataFrame): El DataFrame a procesar.
    """

    # Identificar registros duplicados en 'item_288'
    duplicados = df.duplicated(subset="item_288", keep=False)

    # Crear el campo 'reincidente'
    df["reincidente"] = duplicados.apply(lambda x: "SI" if x else "NO")

    return df

In [72]:
df_2023_filtrado = crear_campo_reincidente(df_2023_filtrado.copy())

df_2023_filtrado.to_csv("df_2023_filtrado_reincidente.csv", index=False)

print("\nDataFrames modificados guardados en archivos CSV.")


DataFrames modificados guardados en archivos CSV.


In [73]:
print("df_2023_filtrado:")
df_2023_filtrado["reincidente"].value_counts()

df_2023_filtrado:


reincidente
NO    8719
SI    4783
Name: count, dtype: int64

## Descripcion binaria | Es o no anomalía

In [74]:
df_2023_filtrado["Descripcion"].unique()

array(['Solo odt', 'Anomalia verificada y aprobada',
       'Anomalia verificada con acta digital', 'Posible anomalia'],
      dtype=object)

In [75]:
def crear_columna_anomalia(df):
    """
    Crea la columna 'Anomalia_conf' en un DataFrame basado en la columna 'Descripcion'.

    Parámetros:
    - df (DataFrame): El DataFrame a procesar.
    """

    def determinar_anomalia(descripcion):
        if descripcion == "Solo odt":
            return "No anomalia"
        else:
            return "Anomalia"

    df["Anomalia_conf"] = df["Descripcion"].apply(determinar_anomalia)
    return df

In [76]:
df_2023_filtrado = crear_columna_anomalia(df_2023_filtrado.copy())

df_2023_filtrado.to_csv("df_2023_filtrado_anomalia.csv", index=False)

print("\nDataFrames modificados guardados en archivos CSV.")


DataFrames modificados guardados en archivos CSV.


In [77]:
print(df_2023_filtrado["Anomalia_conf"].value_counts())

Anomalia_conf
No anomalia    11102
Anomalia        2400
Name: count, dtype: int64


In [78]:
df_2023_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13502 entries, 5 to 66367
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   item_68         13502 non-null  bool   
 1   item_248        13502 non-null  object 
 2   item_74         13502 non-null  bool   
 3   item_597        13502 non-null  object 
 4   item_602        13502 non-null  object 
 5   item_108        13502 non-null  object 
 6   item_43         13502 non-null  object 
 7   item_603        13502 non-null  object 
 8   item_599        13502 non-null  object 
 9   item_35         13502 non-null  object 
 10  item_598        13502 non-null  object 
 11  item_33         13502 non-null  object 
 12  item_601        13502 non-null  object 
 13  item_24         13502 non-null  object 
 14  item_23         13502 non-null  object 
 15  item_237        13502 non-null  bool   
 16  LATI_USU        13502 non-null  float64
 17  LONG_USU        13502 non-null  floa

In [86]:
df_2023_filtrado_py = pd.read_csv(data_processed_dir("dataset_limpio_2023.csv"))
df_2023_filtrado_py[df_2023_filtrado_py.ZONA == 'PACIFICO'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 13502 entries, 5 to 66357
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   item_68         13502 non-null  bool   
 1   item_248        13502 non-null  object 
 2   item_74         13502 non-null  bool   
 3   item_597        13502 non-null  object 
 4   item_602        13502 non-null  object 
 5   item_108        13502 non-null  object 
 6   item_43         13502 non-null  object 
 7   item_603        13502 non-null  object 
 8   item_599        13502 non-null  object 
 9   item_35         13502 non-null  object 
 10  item_598        13502 non-null  object 
 11  item_33         13502 non-null  object 
 12  item_601        13502 non-null  object 
 13  item_24         13502 non-null  object 
 14  item_23         13502 non-null  object 
 15  item_237        13502 non-null  bool   
 16  LATI_USU        13502 non-null  float64
 17  LONG_USU        13502 non-null  floa

In [88]:
df_2023_filtrado_py.head(5)

,item_68,item_248,item_74,item_597,item_602,item_108,item_43,item_603,item_599,item_35,item_598,item_33,item_601,item_24,item_23,item_237,LATI_USU,LONG_USU,NIVEL,AREA,item_288,odt,orden,PLAN_COMERCIAL,Descripcion,reincidente,Anomalia_conf,ZONA
0,False,NO INDICA,True,NOAPL,NOAPL,7.5,376298.0,RA,AO,1301466,AO,1301465.0,DA,NO,SI,True,1.286699,-77.472667,1.0,Urbano,221830,1552391,26490040,2-PLAN RESIDENCIAL,NaN,SI,Anomalia,OCCIDENTE
1,False,NO INDICA,True,NOAPL,NOAPL,9.5,96833.0,RA,AO,1301487,AO,1301488.0,DA,NO,SI,True,1.291291,-77.468665,1.0,Urbano,222838,1552389,26489993,2-PLAN RESIDENCIAL,NaN,NO,Anomalia,OCCIDENTE
2,False,NO INDICA,False,NOAPL,NOAPL,8.0,NO INDICA,NOAPL,VR,3081528,VE,3081527.0,NOAPL,NO,SI,True,1.091228,-77.613528,1.0,Urbano,302820,2185362,28888135,2-PLAN RESIDENCIAL,NaN,NO,Anomalia,SUR
3,False,NO INDICA,True,NOAPL,DA,9.8,678613.0,RA,NOAPL,NO INDICA,BT,1617017.0,DA,SI,SI,True,1.265679,-77.327294,1.0,Rural,8550869,1584217,26546298,2-PLAN RESIDENCIAL,NaN,SI,Anomalia,OCCIDENTE
4,False,INSTALACION ACOM MONOFASICA,False,NOAPL,NOAPL,NO INDICA,404213.0,RA,VR,266906,VR,266905.0,DA,NO,SI,True,1.413125,-77.537665,1.0,Rural,442564,1578384,26589317,2-PLAN RESIDENCIAL,NaN,NO,Anomalia,OCCIDENTE
